## Demo Downloads

https://github.com/sequence-toolbox/QCE-2025-Tutorial

# Star Network Simulation

In this file, we'll demonstrate the simulation of a more complicated network topology with requests between two nodes. These applications will act on each node, first choosing a random other node from the network and then requesting a random number of entangled pairs between the local and distant nodes. The network topology, including hardware components, is shown below:

![network_topo](figures/star_network.png)

## Simulation 1: SeQUeNCe Basics

In this example, we construct the network described above. We'll be building the topology from an external json file `star_network.json`.

### Imports

We must first import the necessary tools from SeQUeNCe.

- `Topology` is a powerful class for creating and managing complex network topologies. We'll be using the `RouterNetTopo` subclass to build our network and intefrace with specific nodes and node types.
- `RequestApp` is an example application included with SeQUeNCe. It will request entanglement with another node in the network at some specified time, using some number of reserved memories on the local node.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact
from sequence.topology.router_net_topo import RouterNetTopo
from sequence.app.request_app import RequestApp

### JSON Specification

We will be using a json file to specify the nodes and connectivity of the network. The json file should be structured as a dictionary with the following keys:

- `is_parallel`, denoting if it’s a parallel or sequential simulation (`False` for this simulation)
- `stop_time`, the stop time of simulation (2 seconds for this simulation)
- `nodes`, giving a list of node specifications,
- One of the following:
    - `qchannels`, giving a list of quantum channel specifications,
    - `qconnections`, giving a list of two-way quantum connection specifications, and
- One of the following:
    - `cchannels`, giving a list of classical channel specifications (similar to qchannels)
    - `cconnections`, giving a list of two-way classical connection specifications
    
The `nodes` field will provide information on the node name, the type of the node, the random number generation seed, and the number of quantum memories. We'll use the `QuantumRouter` node type, which includes the required protocols. Shown here is an example specification for the center node.
```
"nodes": [
    {
      "name": "center",
      "type": "QuantumRouter",
      "seed": 0,
      "memo_size": 50
    },
    ...
]
```

We'll use `qconnections`, which will generate two-way quantum links between nodes. We will specify the end nodes of the link, the optical attenuation, the length, and the type of the link (in this case, `meet_in_the_middle` which will generate a measurement node at the center automatically).
```
"qconnections": [
    {
      "node1": "center",
      "node2": "end1",
      "attenuation": 0.0002,
      "distance": 500,
      "type": "meet_in_the_middle"
    },
    ...
]
```

Finally, for the classical channels, we will be using `cconnections` to create two-way classical links between nodes. We assume the channels have the same delay each way specified by the attribute `delay`.
```
"cconnections": [
    {
      "node1": "center",
      "node2": "end1",
      "delay": 500000000
    },
    ...
]
```

### Building the Simulation

We'll now construct the network and add our applications. This example follows the usual process to ensure that all tools function properly:

1. Create the topology instance for the simulation to manage our network
    - This class will create a simulation timeline
    - This instantiates the Simulation Kernel (see below)
2. Create the simulated network topology. In this case, we are using an external JSON file to specify nodes and their connectivity.
    - This includes specifying hardware parameters in the `set_parameters` function, defined later
    - This instantiates the Hardware, Entanglement Management, Resource Management, and Network Management modules
3. Install custom protocols/applications and ensure all are set up properly (in Example 2)
    - This instantiates the Application module
4. Initialize and run the simulation
    - We will use the `get_timeline` method to access the simulation kernel
5. Collect and display the desired metrics


In [2]:
network_config = "star_network.json"

network_topo = RouterNetTopo(network_config)
tl = network_topo.get_timeline()

# the start and end nodes may be edited as desired
node1_name = "router1"
node2_name = "router2"

node1 = node2 = None
for router in network_topo.get_nodes_by_type(RouterNetTopo.QUANTUM_ROUTER):
    if router.name == node1_name:
        node1 = router
    elif router.name == node2_name:
        node2 = router

# create application to request entanglement
app1 = RequestApp(node1)
app2 = RequestApp(node2)

tl.init()
app1.start(node2_name, start_t=1e12, end_t=2e12, memo_size=25, fidelity=0.9)
tl.run()

For this simulation, the main metric we'll look at is how many memories were entangled using the supplied parameters. We can also look at the state of each memory on node 1 at the time the simulation finished.

In [3]:
# display metrics
print(f"Total memories entangled: {app1.memory_counter}")

print("")
print("Memory entanglement:")
print(node1.name, "memories")
remote_nodes = []
fidelities = []
times = []
for info in node1.resource_manager.memory_manager:
    remote_nodes.append(info.remote_node)
    fidelities.append(info.fidelity)
    times.append(info.entangle_time * 1e-12)
log = {"Remote Node": remote_nodes, "Fidelity": fidelities, "Entanglement Time": times}
df = pd.DataFrame(log)
print(df)

Total memories entangled: 10

Memory entanglement:
router1 memories
   Remote Node  Fidelity  Entanglement Time
0         None  0.000000      -1.000000e-12
1       center  0.850000       1.999013e+00
2      router2  0.867125       1.996254e+00
3         None  0.000000      -1.000000e-12
4         None  0.000000      -1.000000e-12
5         None  0.000000      -1.000000e-12
6      router2  0.831017       1.998259e+00
7         None  0.000000      -1.000000e-12
8      router2  0.899037       1.980493e+00
9         None  0.000000      -1.000000e-12
10        None  0.000000      -1.000000e-12
11        None  0.000000      -1.000000e-12
12      center  0.913403       1.999008e+00
13        None  0.000000      -1.000000e-12
14      center  0.850000       1.999757e+00
15        None  0.000000      -1.000000e-12
16        None  0.000000      -1.000000e-12
17      center  0.884146       1.999257e+00
18      center  0.850000       1.998013e+00
19        None  0.000000      -1.000000e-12
20      

## Simulation 2: Customization

### Setting parameters

Here we define the set_parameters function we used earlier. This function will take a `Topology` as input and change many parameters to desired values. The parameters we can change are shown in the figure below:

![parameters](figures/params.png)

The simulation time limit will be set using the `get_timeline` method.

Quantum memories and detectors are hardware elements, and so parameters are changed by accessing the hardware included with the `QuantumRouter` and `BSMNode` node types. Many complex hardware elements, such as bsm devices or memory arrays, have methods to update parameters for all included hardware elements. This includes `update_memory_params` to change all memories in an array or `update_detector_params` to change all detectors.

We will also set the success probability and swapping degradation of the entanglement swapping protocol. This will be set in the Network management Module (specifically the reservation protocol), as this information is necessary to create and manage the rules for the Resource Management module.

Lastly, we'll update some parameters of the quantum channels. Quantum channels (and, similarly, classical channels) can be accessed from the `Topology` class using the `get_qchannels` method. Since these are individual hardware elements, we will set the parameters directly.

In [4]:
def set_parameters(topology, simulation_time, attenuation):
    """
    simulation_time: duration of simulation time (in ms)
    attenuation: attenuation on quantum channels (in db/m)
    """

    PS_PER_S = 1e12

    # set timeline stop time
    topology.get_timeline().stop_time = (simulation_time * PS_PER_S)

    # set memory parameters
    MEMO_FREQ = 2e2
    MEMO_EXPIRE = 0
    MEMO_EFFICIENCY = 1
    MEMO_FIDELITY = 0.9349367588934053
    for node in topology.get_nodes_by_type(RouterNetTopo.QUANTUM_ROUTER):
        memory_array = node.get_components_by_type("MemoryArray")[0]
        memory_array.update_memory_params("frequency", MEMO_FREQ)
        memory_array.update_memory_params("coherence_time", MEMO_EXPIRE)
        memory_array.update_memory_params("efficiency", MEMO_EFFICIENCY)
        memory_array.update_memory_params("raw_fidelity", MEMO_FIDELITY)

    # set detector parameters
    DETECTOR_EFFICIENCY = 0.9
    DETECTOR_COUNT_RATE = 5e7
    DETECTOR_RESOLUTION = 100
    for node in topology.get_nodes_by_type(RouterNetTopo.BSM_NODE):
        bsm = node.get_components_by_type("SingleAtomBSM")[0]
        bsm.update_detectors_params("efficiency", DETECTOR_EFFICIENCY)
        bsm.update_detectors_params("count_rate", DETECTOR_COUNT_RATE)
        bsm.update_detectors_params("time_resolution", DETECTOR_RESOLUTION)

    # set entanglement swapping parameters
    SWAP_SUCC_PROB = 0.90
    SWAP_DEGRADATION = 0.99
    for node in topology.get_nodes_by_type(RouterNetTopo.QUANTUM_ROUTER):
        node.network_manager.protocol_stack[1].set_swapping_success_rate(SWAP_SUCC_PROB)
        node.network_manager.protocol_stack[1].set_swapping_degradation(SWAP_DEGRADATION)

    # set quantum channel parameters
    ATTENUATION = attenuation
    QC_FREQ = 1e11
    for qc in topology.qchannels:
        qc.attenuation = ATTENUATION
        qc.frequency = QC_FREQ

### Adjusting the application

We will also make our own application to collect some custom metrics. Specifically, we will modify the `get_memory` method to record the time at which entangled pairs are generated.

In [5]:
class ModifiedRequestApp(RequestApp):
    def __init__(self, node):
        super().__init__(node)
        self.entangle_times = []
        
    def get_memory(self, info):
        super().get_memory(info)
        curr_time = self.node.timeline.now()
        self.entangle_times.append(curr_time)

Finally, we'll run all of our code and observe the results under various parameter regimes.

In [6]:
def test(sim_time, qc_atten, min_fidel):
    """
    sim_time: simulation time (in s)
    qc_atten: quantum channel attenuation (dB/km)
    min_fidel: minimum requested fidelity of entanglement
    """
    network_topo = RouterNetTopo(network_config)
    tl = network_topo.get_timeline()

    set_parameters(network_topo, sim_time, qc_atten)

   # the start and end nodes may be edited as desired
    node1_name = "router1"
    node2_name = "router2"
    
    node1 = node2 = None
    for router in network_topo.get_nodes_by_type(RouterNetTopo.QUANTUM_ROUTER):
        if router.name == node1_name:
            node1 = router
        elif router.name == node2_name:
            node2 = router
    
    # create application to request entanglement
    app1 = ModifiedRequestApp(node1)
    app2 = ModifiedRequestApp(node2)
    
    tl.init()
    app1.start(node2_name, start_t=1e12, end_t=2e12, memo_size=25, fidelity=min_fidel)
    tl.run()

    # display metrics
    print("Memory entanglement:")
    print(node1.name, "memories")
    remote_nodes = []
    fidelities = []
    times = []
    for info in node1.resource_manager.memory_manager:
        remote_nodes.append(info.remote_node)
        fidelities.append(info.fidelity)
        times.append(info.entangle_time * 1e-12)
    log = {"Remote Node": remote_nodes, "Fidelity": fidelities, "Entanglement Time": times}
    df = pd.DataFrame(log)
    print(df)
    
    times = np.array(app1.entangle_times) / 1e12  # convert from ps to s
    cumulative_memories = np.linspace(1, app1.memory_counter, app1.memory_counter)
    plt.plot(times, cumulative_memories, marker='o')
    plt.xlabel('Simulation Time (s)')
    plt.ylabel('Number of Entangled Memories')
    plt.show()

In [7]:
interact(test, sim_time=1.5, qc_atten=[0, 2e-5, 2e-4], min_fidel=(0.9, 0.99, 0.01))

interactive(children=(FloatSlider(value=1.5, description='sim_time', max=4.5, min=-1.5), Dropdown(description=…

<function __main__.test(sim_time, qc_atten, min_fidel)>